In [135]:
# imports
import os
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
import keras
import cv2
import numpy as np
import matplotlib.pyplot as plt
import PIL
from PIL import Image

In [136]:
def clean_image(img):
    image = img.copy()
    _,thresh = cv2.threshold(image,0,255,cv2.THRESH_BINARY)
    ctrs,hierarchy= cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    # sorted by width from small to large
    
    # exceptions
    try: hierarchy = hierarchy[0]
    except: hierarchy = []
    
    
    new_rects_ = []
    print(len(ctrs))
    if (len(ctrs))>1:
        
        ctrs = sorted(ctrs, key = lambda ctr:cv2.boundingRect(ctr)[2])
        for contour in range(len(ctrs)-1):
            (x,y,w,h) = cv2.boundingRect(ctrs[contour])

            new_rects_.append([x,y,w,h])
            cv2.rectangle(image, (x,y), (x+w,y+h), (255, 0, 0), 2) 

        for i in range(len(new_rects_)):
            x =  new_rects_[i][0]
            y =  new_rects_[i][1]
            w =  new_rects_[i][2]
            h =  new_rects_[i][3]
            start_point = (x-3, y-3)
            end_point = (x+w+5, y+h+3)
            color = (0, 0, 0)
            thickness = -1
            image = cv2.rectangle(image, start_point, end_point, color, thickness)       
            
            
    # change image shape
    h, w = image.shape
    if w>=h:
        outputImage = cv2.copyMakeBorder(image,int(w/2)+10-int(h/2),int(w/2)+10-int(h/2),10,10,cv2.BORDER_CONSTANT,value=[0, 0, 0])
    if h>w:
        outputImage = cv2.copyMakeBorder(image,10,10,int(h/2)+10-int(w/2),int(h/2)+10-int(w/2),cv2.BORDER_CONSTANT,value=[0, 0, 0])

    return outputImage


In [137]:
def maintain_aspect_ratio_resize(image, width=None, height=None, inter=cv2.INTER_AREA):
    # Grab the image size and initialize dimensions
    dim = None
    (h, w) = image.shape
    
    # Return original image if no need to resize
    if width is None and height is None:
        return image

    # We are resizing height if width is none
    if width is None:
        # Calculate the ratio of the height and construct the dimensions
        r = height / float(h)
        dim = (int(w * r), height)
    # We are resizing width if height is none
    else:
        # Calculate the ratio of the width and construct the dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # Return the resized image
    return cv2.resize(image, dim, interpolation=inter)

In [ ]:
########################################################################################################################
def get_rows(img, centers, row_amt, row_h):
    centers = np.array(centers)
    d = row_h / row_amt
    for i in range(row_amt):
        f = centers[:, 1] - d * i
        a = centers[(f < d) & (f > 0)]
        yield a[a.argsort(0)[:, 0]]
        

############################################################################################################################

id_ = []
rects_sorted = []
new_rects4 = []
center = []
arr_id = []

def row_funct(img,h):
    global train_data
    count = 0
    train_data = []
    h1, w = img.shape
    for row in get_rows(img, center, h, h1):
        cv2.polylines(img, [row], False, (255, 0, 255), 2)
        for x, y in row:
            thresh_id = int(str(x)+str(y))
            id_.append(thresh_id)
            count += 1
            cv2.circle(img, (x, y), 10, (0, 0, 255), -1)  
            cv2.putText(img, str(count), (x - 10, y + 5), 1, cv2.FONT_HERSHEY_PLAIN, (255, 255, 255), 2)

    plt.imshow(img, cmap="gray") 
    plt.show()


    ###########################################################################################################################

    # sorted rects 

    for f in id_:
        ind = arr_id.index(f)
        r = new_rects4[ind]
        rects_sorted.append(r)
        x = r[0]
        y = r[1]
        w = r[2]
        h = r[3]
        
        im_crop = thresh[y:y+h,x:x+w]
        if w>2*h:
            kernel = np.ones((2,2), np.uint8)
            im_crop = thresh[y:y+h,x:x+int(w/2.3)]
            #im_crop = cv2.dilate(im_crop, kernel, iterations=3)

        out = clean_image(im_crop)
        kernel = np.ones((2,2), np.uint8)
        resize_ratio = 0.1

        out = maintain_aspect_ratio_resize(out, width=int(40))
        #out = cv2.resize(out,(28,28))             
        out = cv2.resize(out,(28,28))

        train_data.append(out)
        plt.imshow(out, cmap="gray") 
        plt.show()
    ###########################################################################################################################

    plt.imshow(img, cmap="gray") 
    plt.show()
    
    count = 0
    return train_data
       

#######################################################################################################################
# Testdata8/eq3


def hamex(imag):
    global thresh, branch,len_eq
    img = ~ imag # invert image colours
    #_,thresh = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
    se=cv2.getStructuringElement(cv2.MORPH_RECT , (8,8))
    bg=cv2.morphologyEx(imag, cv2.MORPH_DILATE, se)
    out_gray=cv2.divide(imag, bg, scale=255)
    out_binary=cv2.threshold(out_gray, 0, 255, cv2.THRESH_OTSU )[1] 
    thresh = ~ out_binary
    out = thresh.copy
    plt.imshow(thresh, cmap="gray") 
    plt.show()  

    ctrs,hierarchy= cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    # sorting by y
    ctrs = sorted(ctrs, key = lambda ctr:cv2.boundingRect(ctr)[1])
    # sorting by x
    ctrs = sorted(ctrs, key = lambda ctr:cv2.boundingRect(ctr)[0])

    try: hierarchy = hierarchy[0]
    except: hierarchy = []
    # computes the bounding box for the contour, and draws it on the frame,


    for contour, hier in zip(ctrs, hierarchy):
        (x,y,w,h) = cv2.boundingRect(contour)
        if w > 4 and h > 4:
            midpoint = [x+w/2,y+h/2]
            area = (w)*(h)
            new_rects4.append([x,y,w,h,midpoint,area])
            # replace thresh with img
            cv2.rectangle(thresh, (x-10,y-10), (x+w+7,y+h+5), (255, 0, 0), 2)        
            points = (x,y)
            arr_id.append(int(str(x)+str(y)))
            center.append(points)

    print(arr_id )  


    train_data = row_funct(thresh,branch)
    len_eq = len(train_data)

    # replace thresh with img
    cv2.imshow('contours',thresh)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
branch = 1
# Testdata1/eq7
imag = cv2.imread('C:/Users/nizen/conda-envs/Final year project/0to9regAndsigns/Testdata5/eq4.jpg',cv2.IMREAD_GRAYSCALE)
hamex(imag)
print(rects_sorted)


In [ ]:

new_model = tf.keras.models.load_model('saved_model16/my_model16')

minus_arr = []
results_arr = []
sqrt_store = []
def predictions(train_data):
    for i in range(len_eq):
        train_data[i] = np.array(train_data[i])
        train_data[i] = train_data[i].reshape(1,28,28,1)
        output_pred = new_model.predict_classes(train_data[i])

        if output_pred[0] == 0:
            results_arr.append("0")
            
        if output_pred[0] == 1:
            if rects_sorted[i][2] > 2*rects_sorted[i][3]:                
                results_arr.append("-") 
                minus_arr.append(i)
            else:
                results_arr.append("1") 

        if output_pred[0] == 2:
            results_arr.append("2")

        if output_pred[0] == 3:
            results_arr.append("3")
            
        if output_pred[0] == 4:
            results_arr.append("4")
            
        if output_pred[0] == 5:
            results_arr.append("5")
                
        if output_pred[0] == 6:
            results_arr.append("6")
            
        if output_pred[0] == 7:
            results_arr.append("7")

        if output_pred[0] == 8:
            results_arr.append("8")

        if output_pred[0] == 9:
            results_arr.append("9")
            
        if output_pred[0] == 10:
            minus_arr.append(i)
            results_arr.append("-")
            
        if output_pred[0] == 11:
            results_arr.append("+")
            
        if output_pred[0] == 12:
            results_arr.append("x")
    
        if output_pred[0] == 13:
            results_arr.append("(")

        if output_pred[0] == 14:
            results_arr.append(")")

        if output_pred[0] == 15:
            results_arr.append("sqrt")
            sqrt_store.append(i)
            
    print(fraction_check(minus_arr))
    

out = []
eq_arr = []
fraction_point = []
def fraction_check(minus_arr):
    global branch, equ, eqind
    equ = False
    dimen = []
    for minus in minus_arr:
        dimension_check = rects_sorted[minus]
        dimen.append(dimension_check)
    
    for i in range(len(dimen)):
        r = dimen[i]
        x = r[0]
        y = r[1]
        w = r[2]
        h = r[3]
        #-int(w/4)
        im_crop_ = thresh[y-int(w/3.5):y+h+int(w/3.5),x:x+w]
        
        plt.imshow(im_crop_, cmap="gray") 
        plt.show() 

        # get image shape
        h,w = im_crop_.shape
        #print('midpoint at ',int(h/2))

        ctrs_,hierarchy_= cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)

        try: hierarchy_ = hierarchy_[0]
        except: hierarchy_ = []


        num = False 
        den = False 
        for contour, hier in zip(ctrs_, hierarchy_):
            (x,y,w,h1) = cv2.boundingRect(contour)
            cv2.rectangle(thresh, (x,y), (x+w,y+h1), (255, 0, 0), 2) 

            if w > 6 and h > 6:
                if y>int(h/2)+10:
                    num = True
                if y<int(h/2)-10:
                    den = True

        if den == True and num == True:
            out.append([minus_arr[i],'fraction'])
            results_arr[minus_arr[i]] = 'fraction'
            fraction_point.append(minus_arr[i])
            if minus_arr[i]<3:
                print('branch activated')
                branch = 2
            #branch = 2
            #out ='fraction'
        if den == True and num == False:
            eq_arr.append([minus_arr[i],'='])
            equ = True
            eqind = minus_arr[i]
        if den == False and num == True:
            eq_arr.append([minus_arr[i],'='])
            equ = True
            eqind = minus_arr[i]
            

    return out

################################################################################################################################
Area = []
def Area_fun(): 
  
    # area must not be a squareroot
    for i in range(len(new_rects4)):
        Area.append(rects_sorted[i][5])
    for i in sqrt_store:
        Area[i] = 0
    large_area = max(Area) #out[-1]
    for i in range(0,len(Area),1):

        cond = False

        if (results_arr[i])=='1':
            cond = True
        if (results_arr[i])=='-':
            cond = True
        if (results_arr[i])=='sqrt':
            cond = True
        if (results_arr[i])=='+':
            cond = True

        if cond != True:
            if 2.8*Area[i]<large_area: 
                results_arr[i] = '^'+ results_arr[i]

                                  
################################################################################################################################
# equal algorithm
def equal_mani():
    for i in range(len(eq_arr)):
        if i+1 < len(eq_arr):
            if (eq_arr[i][0]+1) == (eq_arr[i+1][0]):
                # replacing      
                results_arr[eq_arr[i][0]] = '='
                results_arr.pop(eq_arr[i][0]+1) 

 

def scan_run():
    predictions(train_data) # runs prediction and fraction check
    Area_fun()
    equal_mani()
    print(results_arr)
    
scan_run()



print(minus_arr)
print(fraction_point)

eq = ''
for i in range(len(results_arr)):
    eq = eq + results_arr[i]
    eqind = eq.find('=')
    
    
    

print('The output equation is ',eq)

In [ ]:
# for squareroot
if equ == True:
    eq = eq[eqind:]

sqr_cont =''
out = ''
for i in range(len(sqrt_store)):
    sq = sqrt_store[i]
    sqr = rects_sorted[sq]
    print(sqr)
    x = sqr[0]
    y = sqr[1]
    w = sqr[2]
    h = sqr[3]

    for i in range(len(rects_sorted)): 
        centroid = rects_sorted[i][4]
        if (x<centroid[0]<x+w) and (y<centroid[1]<y+h) and results_arr[i]!='sqrt':
            sqr_cont = sqr_cont + results_arr[i]
    out = '('+'sqrt('+sqr_cont+')'+')'

print('The output equation is ',sqr_cont,eq)

In [ ]:
# inside

# get fraction point

if equ == True:
    eq = eq[eqind:]
    


frac_in = ''
for i in range(len(fraction_point)):
    rac = fraction_point[i]
    frac = rects_sorted[rac]
    print(frac)
    x = frac[0]
    y = frac[1]
    w = frac[2]
    h = frac[3]

    num = ''
    den = ''
    for i in range(len(rects_sorted)): 

        centroid = rects_sorted[i][4]

        if x<centroid[0]<x+w and (centroid[1]<y):
            num = num + results_arr[i]

        # sotre coordinates numerator then get rowns and sh
        # get rows for this

        if x<centroid[0]<x+w and (centroid[1]>y):
            den = den + results_arr[i]
            
    frac_in = 'frac(' + num+','+ den+')') + eq


print('The output equation is ',frac_in)

